In [1]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score
from scipy import sparse
from datetime import datetime
from catboost import CatBoostRegressor

C:\Users\Evgeny\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
movies_imdb = pd.read_csv('./movies_imdb.csv')

In [3]:
als_train_u0_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_0.csv')
als_train_u0_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_1.csv')
als_train_u0_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_2.csv')
als_train_u1_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_0.csv')
als_train_u1_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_1.csv')
als_train_u1_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_2.csv')
als_train_u2_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_0.csv')
als_train_u2_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_1.csv')
als_train_u2_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_2.csv')

als_test = pd.read_csv('./ALS/ratings_dev_with_all_ALS_predictions.csv')
als_final_test = pd.read_csv('./ALS/ratings_test_with_all_ALS_predictions.csv')

In [4]:
a = pd.read_csv('./predictions_linear/predictions_0', header=None)

In [5]:
print(a.shape)
print(als_train_u0_m0.shape)
print(a.head(3))
print(als_train_u0_m0.head(3))

(1567557, 5)
(1567557, 7)
   0       1   2          3         4
0  0   53434  19  822873600  2.558093
1  0  124035  47  823185196  3.624575
2  0  124035  52  823185197  3.384482
   movieId  userId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0    33794  108543  1141315698     4.0        4.061448        3.435761   
1    33794  116656  1308349074     4.5        4.055828        3.234410   
2    33794  113116  1297394692     3.0        3.999297        3.608127   

   user_actor_ALS  
0        3.240306  
1        3.690494  
2        3.915428  


In [6]:
lin_features_0 = pd.read_csv('./predictions_linear/predictions_0', header=None)
lin_features_1 = pd.read_csv('./predictions_linear/predictions_1', header=None)
lin_features_2 = pd.read_csv('./predictions_linear/predictions_2', header=None)
lin_features_3 = pd.read_csv('./predictions_linear/predictions_3', header=None)
lin_features_4 = pd.read_csv('./predictions_linear/predictions_4', header=None)
lin_features_5 = pd.read_csv('./predictions_linear/predictions_5', header=None)
lin_features_6 = pd.read_csv('./predictions_linear/predictions_6', header=None)
lin_features_7 = pd.read_csv('./predictions_linear/predictions_7', header=None)
lin_features_8 = pd.read_csv('./predictions_linear/predictions_8', header=None)

In [7]:
lin_features_0.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_1.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_2.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_3.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_4.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_5.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_6.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_7.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_8.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)

In [8]:
print(lin_features_1.shape)
print(als_train_u0_m1.shape)

(1544890, 5)
(1544890, 7)


In [9]:
lin_features_0.head(2)

,fold,userId,movieId,timestamp,lin_pred
0,0,53434,19,822873600,2.558093
1,0,124035,47,823185196,3.624575


In [10]:
als_train_u0_m0 = pd.merge(als_train_u0_m0, lin_features_0, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u0_m1 = pd.merge(als_train_u0_m1, lin_features_1, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u0_m2 = pd.merge(als_train_u0_m2, lin_features_2, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m0 = pd.merge(als_train_u1_m0, lin_features_3, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m1 = pd.merge(als_train_u1_m1, lin_features_4, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m2 = pd.merge(als_train_u1_m2, lin_features_5, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m0 = pd.merge(als_train_u2_m0, lin_features_6, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m1 = pd.merge(als_train_u2_m1, lin_features_7, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m2 = pd.merge(als_train_u2_m2, lin_features_8, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])

In [11]:
als_train_u1_m1.head(2)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred
0,6502,10416,1127117051,4.0,3.706461,3.368054,NaN,4,3.625825
1,24,23457,1205769764,5.0,3.173135,3.481067,3.033822,4,3.106729


In [12]:
als_train = pd.concat([als_train_u0_m0, als_train_u0_m1, als_train_u0_m2,
                       als_train_u1_m0, als_train_u1_m1, als_train_u1_m2,
                       als_train_u2_m0, als_train_u2_m1, als_train_u2_m2],
                      ignore_index=True)

In [13]:
movies_imdb.head(3)

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_imdb,movieId,title,genres,titleCleaned,year,nconst,averageRating,numVotes
0,0,tt0066856,movie,born to win,born to win,0,1971,\N,88,"Comedy,Crime,Drama",3212,Born to Win (1971),Drama,born to win,1971,nm0005837nm0001719nm0696038nm0000947nm0282031n...,5.5,1432.0
1,1,tt0066858,movie,boy friend,boy friend,0,1971,\N,137,"Comedy,Musical,Romance",74859,"Boy Friend, The (1971)",Comedy|Musical|Romance,boy friend,1971,nm0910589nm0493341nm0300083nm0012451nm0697799n...,6.8,2764.0
2,2,tt0066892,movie,crnl knowledge,crnl knowledge,0,1971,\N,98,Drama,3167,Carnal Knowledge (1971),Comedy|Drama,crnl knowledge,1971,nm0000197nm0000298nm0000268nm0307316nm0001566n...,7.0,10324.0


In [14]:
print(movies_imdb.shape)
print(movies_imdb['movieId'].nunique())

(19636, 18)
19636


In [15]:
movies_imdb['averageRating'] = movies_imdb['averageRating'] / 2

In [16]:
genres = list(map(lambda x: [el for el in x.split(',')] + ['None'] * (3 - len(x.split(','))), movies_imdb['genres_imdb']))

In [17]:
movies_imdb['genres_imdb_0'] = [el[0] for el in genres]
movies_imdb['genres_imdb_1'] = [el[1] for el in genres]
movies_imdb['genres_imdb_2'] = [el[2] for el in genres]

In [18]:
sum(movies_imdb['year'].isna())

0

In [19]:
movies_imdb['runtimeMinutes'][movies_imdb['runtimeMinutes'] == '\\N'] = -1

C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
movies_imdb.head(3)

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_imdb,...,title,genres,titleCleaned,year,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2
0,0,tt0066856,movie,born to win,born to win,0,1971,\N,88,"Comedy,Crime,Drama",...,Born to Win (1971),Drama,born to win,1971,nm0005837nm0001719nm0696038nm0000947nm0282031n...,2.75,1432.0,Comedy,Crime,Drama
1,1,tt0066858,movie,boy friend,boy friend,0,1971,\N,137,"Comedy,Musical,Romance",...,"Boy Friend, The (1971)",Comedy|Musical|Romance,boy friend,1971,nm0910589nm0493341nm0300083nm0012451nm0697799n...,3.40,2764.0,Comedy,Musical,Romance
2,2,tt0066892,movie,crnl knowledge,crnl knowledge,0,1971,\N,98,Drama,...,Carnal Knowledge (1971),Comedy|Drama,crnl knowledge,1971,nm0000197nm0000298nm0000268nm0307316nm0001566n...,3.50,10324.0,Drama,None,None


In [21]:
print(als_train.shape)
print(als_test.shape)
print(als_train.head(2))
print(als_test.head(2))
print(als_final_test.shape)
print(als_final_test.head(2))

(14453730, 9)
(1690360, 7)
   movieId  userId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0    33794  108543  1141315698     4.0        4.061448        3.435761   
1    33794  116656  1308349074     4.5        4.055828        3.234410   

   user_actor_ALS  fold  lin_pred  
0        3.240306     0  3.770295  
1        3.690494     0  3.869308  
   userId  movieId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0   40965     1217  1180039623     5.0        3.616524        3.165045   
1  131172      466   835337608     4.0        2.412944        3.676014   

   user_actor_ALS  
0        3.187873  
1        2.545692  
(3790860, 6)
   userId  movieId   timestamp  user_movie_ALS  user_genre_ALS  user_actor_ALS
0    2711     5902  1044313557        4.242603        3.521633             NaN
1    3352     1639   895849643        3.059602        3.224138        3.300966


In [22]:
def rmse_asymmetric(rating, prediction):
    bad_border = 4
    
    if rating < bad_border <= prediction:
        penalty_coefficient = 2.0
    else:
        penalty_coefficient = 1.0
    
    return penalty_coefficient*(rating - prediction)**2


def vector_rmse_asymmetric(df, true_col, pred_col, bad_border=4):
    data = np.array(df[[true_col, pred_col]].values)
    mask_1 = data[:, 0] < bad_border
    mask_2 = data[:, 1] >= bad_border
    penalty = data[mask_1 & mask_2]
    not_penalty = data[~mask_1 & mask_2]
    result = np.mean(np.concatenate((2 * (penalty[:, 0] - penalty[:, 1])  ** 2,
                                    (not_penalty[:, 0] - not_penalty[:, 1])  ** 2), 
                                    axis=None))
    return result

In [23]:
als_train = pd.merge(als_train, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])
als_test = pd.merge(als_test, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])
als_final_test = pd.merge(als_final_test, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])

In [24]:
als_train['titleType'].fillna('None', inplace=True)
als_test['titleType'].fillna('None', inplace=True)
als_final_test['titleType'].fillna('None', inplace=True)

als_train['genres_imdb_0'].fillna('None', inplace=True)
als_test['genres_imdb_0'].fillna('None', inplace=True)
als_final_test['genres_imdb_0'].fillna('None', inplace=True)

als_train['genres_imdb_1'].fillna('None', inplace=True)
als_test['genres_imdb_1'].fillna('None', inplace=True)
als_final_test['genres_imdb_1'].fillna('None', inplace=True)

als_train['genres_imdb_2'].fillna('None', inplace=True)
als_test['genres_imdb_2'].fillna('None', inplace=True)
als_final_test['genres_imdb_2'].fillna('None', inplace=True)

In [25]:
als_train.head(3)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred,Unnamed: 0,...,title,genres,titleCleaned,year,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2
0,33794,108543,1141315698,4.0,4.061448,3.435761,3.240306,0,3.770295,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None
1,33794,116656,1308349074,4.5,4.055828,3.234410,3.690494,0,3.869308,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None
2,33794,113116,1297394692,3.0,3.999297,3.608127,3.915428,0,3.770065,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None


In [26]:
als_test.columns

Index(['userId', 'movieId', 'timestamp', 'rating', 'user_movie_ALS',
       'user_genre_ALS', 'user_actor_ALS', 'Unnamed: 0', 'tconst', 'titleType',
       'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear',
       'runtimeMinutes', 'genres_imdb', 'title', 'genres', 'titleCleaned',
       'year', 'nconst', 'averageRating', 'numVotes', 'genres_imdb_0',
       'genres_imdb_1', 'genres_imdb_2'],
      dtype='object')

In [27]:
als_train['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_train['timestamp']))
als_test['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_test['timestamp']))
als_final_test['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_final_test['timestamp']))

In [28]:
als_train['curr_year'] = list(map(lambda x: float(x[:4]), als_train['curr_date']))
als_test['curr_year'] = list(map(lambda x: float(x[:4]), als_test['curr_date']))
als_final_test['curr_year'] = list(map(lambda x: float(x[:4]), als_final_test['curr_date']))

als_train['curr_month'] = list(map(lambda x: x[5:7], als_train['curr_date']))
als_test['curr_month'] = list(map(lambda x: x[5:7], als_test['curr_date']))
als_final_test['curr_month'] = list(map(lambda x: x[5:7], als_final_test['curr_date']))

als_train['movie_age'] = als_train['curr_year'] - als_train['year']
als_test['movie_age'] = als_test['curr_year'] - als_test['year']
als_final_test['movie_age'] = als_final_test['curr_year'] - als_final_test['year']

In [29]:
als_train.head(1)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred,Unnamed: 0,...,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2,curr_date,curr_year,curr_month,movie_age
0,33794,108543,1141315698,4.0,4.061448,3.435761,3.240306,0,3.770295,9943.0,...,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None,2006-03-02T19:08:18,2006.0,03,1.0


In [30]:
print(vector_rmse_asymmetric(als_train, 'rating', 'user_movie_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'user_genre_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'user_actor_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'averageRating'))
print(vector_rmse_asymmetric(als_train, 'rating', 'lin_pred'))

C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


0.6197099090875856
1.0423400236647804
0.7162128806061265
1.3887644861972723
1.4576650118188108


In [44]:
feature_columns = [
    'user_movie_ALS', 
    'user_genre_ALS', 
    'user_actor_ALS', 
    'averageRating', 
    'isAdult', 
    #'runtimeMinutes', 
    'numVotes', 
    #'year', 
    'movie_age', 
    #'curr_year', 
    #'lin_pred'
]
cat_feature_columns = [
    #'titleType', 
    #'genres_imdb_0', 
    #'genres_imdb_1', 
    #'genres_imdb_2', 
    #'curr_month'
]
train_columns = feature_columns + cat_feature_columns

In [45]:
train_data = als_train[train_columns]
train_target = als_train['rating']

test_data = als_test[train_columns]
test_target = als_test['rating']

In [46]:
bad_border = 4
mask_1 = np.array(train_target.values) < bad_border
weights = np.ones(len(train_data))
weights[mask_1] = 2

In [47]:
weights

array([1., 1., 2., ..., 2., 1., 2.])

In [48]:
model = CatBoostRegressor(iterations=200)

In [49]:
model.fit(train_data, train_target, verbose=True, sample_weight=weights, cat_features=cat_feature_columns)

0:	learn: 3.3415009	total: 1.12s	remaining: 3m 42s
1:	learn: 3.2473467	total: 2.17s	remaining: 3m 35s
2:	learn: 3.1562172	total: 3.3s	remaining: 3m 36s
3:	learn: 3.0681277	total: 4.4s	remaining: 3m 35s
4:	learn: 2.9826224	total: 5.45s	remaining: 3m 32s
5:	learn: 2.8998762	total: 6.5s	remaining: 3m 30s
6:	learn: 2.8198332	total: 7.49s	remaining: 3m 26s
7:	learn: 2.7422912	total: 8.51s	remaining: 3m 24s
8:	learn: 2.6672411	total: 9.53s	remaining: 3m 22s
9:	learn: 2.5946420	total: 10.6s	remaining: 3m 20s
10:	learn: 2.5245620	total: 11.6s	remaining: 3m 18s
11:	learn: 2.4567245	total: 12.6s	remaining: 3m 17s
12:	learn: 2.3912064	total: 13.8s	remaining: 3m 18s
13:	learn: 2.3278381	total: 14.9s	remaining: 3m 17s
14:	learn: 2.2665473	total: 16s	remaining: 3m 17s
15:	learn: 2.2072970	total: 17.1s	remaining: 3m 16s
16:	learn: 2.1500372	total: 18.3s	remaining: 3m 16s
17:	learn: 2.0947409	total: 19.4s	remaining: 3m 16s
18:	learn: 2.0414552	total: 20.6s	remaining: 3m 16s
19:	learn: 1.9898041	total:

157:	learn: 0.7963548	total: 2m 57s	remaining: 47.1s
158:	learn: 0.7963154	total: 2m 58s	remaining: 45.9s
159:	learn: 0.7962776	total: 2m 59s	remaining: 44.8s
160:	learn: 0.7962397	total: 3m	remaining: 43.7s
161:	learn: 0.7962050	total: 3m 1s	remaining: 42.6s
162:	learn: 0.7961703	total: 3m 2s	remaining: 41.4s
163:	learn: 0.7961390	total: 3m 3s	remaining: 40.3s
164:	learn: 0.7961108	total: 3m 4s	remaining: 39.2s
165:	learn: 0.7960827	total: 3m 5s	remaining: 38.1s
166:	learn: 0.7960570	total: 3m 6s	remaining: 36.9s
167:	learn: 0.7960299	total: 3m 8s	remaining: 35.8s
168:	learn: 0.7960053	total: 3m 9s	remaining: 34.7s
169:	learn: 0.7959797	total: 3m 10s	remaining: 33.6s
170:	learn: 0.7959580	total: 3m 11s	remaining: 32.5s
171:	learn: 0.7959355	total: 3m 12s	remaining: 31.3s
172:	learn: 0.7959175	total: 3m 13s	remaining: 30.2s
173:	learn: 0.7958953	total: 3m 14s	remaining: 29.1s
174:	learn: 0.7958769	total: 3m 15s	remaining: 28s
175:	learn: 0.7958597	total: 3m 16s	remaining: 26.8s
176:	le

In [50]:
catboost_predict = model.predict(test_data)

In [51]:
als_test['catboost_predict'] = catboost_predict

In [52]:
print(vector_rmse_asymmetric(als_test, 'rating', 'catboost_predict'))

0.6169683348334891


In [53]:
model.feature_importances_

array([9.75151179e+01, 8.65068934e-01, 4.80185482e-01, 2.85068507e-01,
       2.82403882e-02, 4.54528438e-01, 3.71790335e-01])

In [ ]:
als_test['mean_predict'] = np.mean(test_data.values, axis=1)

In [ ]:
print(vector_rmse_asymmetric(als_test, 'rating', 'mean_predict'))

In [ ]:
als_test['median_predict'] = np.median(test_data.values, axis=1)

In [ ]:
print(vector_rmse_asymmetric(als_test, 'rating', 'median_predict'))

In [ ]:
final_test_results = model.predict(als_final_test[train_columns])

In [ ]:
als_final_test['prediction'] = final_test_results

In [ ]:
als_final_test.to_csv('./prediction.csv')

In [ ]:
import pickle

In [ ]:
with open('./saved_catboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)